In [ ]:
import pandas as pd
import numpy as np
base_casas = pd.read_csv('house_prices.csv')
base_casas

In [ ]:
base_casas.drop(columns = ['id', 'date', 'sqft_living15', 'sqft_lot15'], axis = 1, inplace = True)
base_casas

In [ ]:
base_treinamento = base_casas.iloc[0:15129,:]
base_treinamento.shape

In [ ]:
base_teste = base_casas.iloc[15129:,:]
base_teste.shape

In [ ]:
X_teste = base_teste.iloc[:,1:17].values
X_teste

In [ ]:
y_teste = base_teste.iloc[:, 0].values
y_teste

In [ ]:
base_treinamento.to_csv('house_prices_train_xgboost.csv', header = False, index = False)
base_teste.to_csv('house_prices_test_xgboost.csv', header = False, index = False)